In [29]:
import pandas as pd 
import numpy as np

import pickle

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from  sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,f1_score



from catboost import CatBoostClassifier

In [2]:
df= pd.read_csv("sensor_raw.csv")

In [7]:
df_train,df_test= train_test_split(df,random_state=2021,test_size=0.2)

In [8]:
df_train

,Target(Class),GyroX,GyroY,GyroZ,AccX,AccY,AccZ
619,3,-7.473282,8.534351,9.732824,0.193115,0.084717,-0.912842
201,1,-2.221374,3.221374,-1.809160,-0.013916,0.125244,-1.136475
998,4,0.664122,4.160305,0.778626,0.506592,-0.243164,-0.920410
833,3,-1.030534,-1.885496,0.274809,0.280029,-0.263428,-1.130371
408,2,0.290076,5.045802,1.969466,0.356445,-0.107666,-1.001221
...,...,...,...,...,...,...,...
669,3,-3.648855,6.122137,2.267176,0.352539,-0.130127,-1.096680
830,3,-2.290076,-1.419847,-0.801527,0.284424,-0.199951,-1.007080
44,1,-0.969466,9.557252,-0.671756,0.168945,-0.155273,-1.114746
621,3,1.145038,4.541985,38.114504,0.275391,0.477051,-0.954590


In [9]:
df_test

,Target(Class),GyroX,GyroY,GyroZ,AccX,AccY,AccZ
133,1,1.648855,4.549618,4.419847,0.088623,0.049561,-0.881836
113,1,6.732824,5.786260,2.396947,0.445312,-0.170654,-0.903076
1070,4,-1.122137,3.786260,0.190840,0.272217,-0.096924,-0.975342
275,2,2.992366,5.763359,1.358779,0.191406,-0.055420,-1.040527
952,4,3.099237,7.450382,0.938931,0.587646,-0.227295,-0.912354
...,...,...,...,...,...,...,...
627,3,-0.099237,2.221374,-1.244275,0.287598,-0.165283,-0.820312
210,1,-14.648855,-0.122137,4.946565,0.049316,-0.014404,-0.794922
1058,4,0.488550,5.969466,1.541985,0.000977,0.057129,-0.848145
235,1,3.946565,11.190840,2.167939,0.615967,-0.083740,-1.140137


In [11]:
df_train['Target(Class)'].value_counts(),df_test['Target(Class)'].value_counts()

(3    285
 2    230
 1    198
 4    178
 Name: Target(Class), dtype: int64,
 3    65
 2    58
 1    54
 4    46
 Name: Target(Class), dtype: int64)

In [12]:
df_test.to_csv("test.csv",index=False)
df_train.to_csv("train.csv",index=False)

In [19]:
X_train = df_train.iloc[:,1:]
y_train = df_train.iloc[:,:1]

X_test = df_test.iloc[:,1:]
y_test = df_test.iloc[:,:1]

In [20]:
#normalizing
scaler=MinMaxScaler(feature_range=(-1,1))

X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [22]:
#random forest
rfc= RandomForestClassifier()
rfc.fit(X_train_scaled,y_train)
y_pred=rfc.predict(X_test_scaled)

print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

<ipython-input-22-26773caf6c63>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train_scaled,y_train)


0.6547085201793722
              precision    recall  f1-score   support

           1       0.63      0.50      0.56        54
           2       0.67      0.66      0.66        58
           3       0.56      0.68      0.61        65
           4       0.84      0.80      0.82        46

    accuracy                           0.65       223
   macro avg       0.67      0.66      0.66       223
weighted avg       0.66      0.65      0.65       223



In [25]:
filename = 'random_forest.sav'
pickle.dump(rfc, open(filename, 'wb'))

In [28]:
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [50]:
#catboost 

cfc = CatBoostClassifier(learning_rate=0.1,
                         iterations=5000
                        )
cfc.fit(X_train_scaled,y_train)

y_pred=cfc.predict(X_test_scaled)

print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

0:	learn: 1.3289610	total: 5.99ms	remaining: 29.9s
1:	learn: 1.2648535	total: 10.6ms	remaining: 26.6s
2:	learn: 1.2161109	total: 15ms	remaining: 25s
3:	learn: 1.1780822	total: 19.5ms	remaining: 24.4s
4:	learn: 1.1454238	total: 24ms	remaining: 23.9s
5:	learn: 1.1073469	total: 28.4ms	remaining: 23.6s
6:	learn: 1.0713451	total: 33.6ms	remaining: 23.9s
7:	learn: 1.0392053	total: 37.9ms	remaining: 23.7s
8:	learn: 1.0114554	total: 43.4ms	remaining: 24.1s
9:	learn: 0.9916138	total: 48.3ms	remaining: 24.1s
10:	learn: 0.9708375	total: 52.6ms	remaining: 23.8s
11:	learn: 0.9527210	total: 57.1ms	remaining: 23.7s
12:	learn: 0.9378080	total: 62.1ms	remaining: 23.8s
13:	learn: 0.9225047	total: 66.4ms	remaining: 23.7s
14:	learn: 0.9065351	total: 70.9ms	remaining: 23.6s
15:	learn: 0.8936544	total: 75.3ms	remaining: 23.5s
16:	learn: 0.8818274	total: 79.3ms	remaining: 23.3s
17:	learn: 0.8702569	total: 84.3ms	remaining: 23.3s
18:	learn: 0.8619776	total: 89.1ms	remaining: 23.3s
19:	learn: 0.8512021	total: 

173:	learn: 0.3932986	total: 864ms	remaining: 24s
174:	learn: 0.3924968	total: 872ms	remaining: 24s
175:	learn: 0.3911538	total: 879ms	remaining: 24.1s
176:	learn: 0.3901717	total: 884ms	remaining: 24.1s
177:	learn: 0.3884099	total: 891ms	remaining: 24.1s
178:	learn: 0.3875250	total: 896ms	remaining: 24.1s
179:	learn: 0.3863592	total: 903ms	remaining: 24.2s
180:	learn: 0.3850182	total: 911ms	remaining: 24.2s
181:	learn: 0.3839562	total: 918ms	remaining: 24.3s
182:	learn: 0.3826330	total: 926ms	remaining: 24.4s
183:	learn: 0.3814793	total: 931ms	remaining: 24.4s
184:	learn: 0.3799500	total: 938ms	remaining: 24.4s
185:	learn: 0.3781995	total: 942ms	remaining: 24.4s
186:	learn: 0.3769171	total: 946ms	remaining: 24.4s
187:	learn: 0.3750946	total: 952ms	remaining: 24.4s
188:	learn: 0.3740479	total: 958ms	remaining: 24.4s
189:	learn: 0.3726411	total: 965ms	remaining: 24.4s
190:	learn: 0.3701835	total: 972ms	remaining: 24.5s
191:	learn: 0.3686273	total: 981ms	remaining: 24.6s
192:	learn: 0.36

349:	learn: 0.2322332	total: 2.24s	remaining: 29.8s
350:	learn: 0.2313817	total: 2.25s	remaining: 29.8s
351:	learn: 0.2309137	total: 2.26s	remaining: 29.8s
352:	learn: 0.2305292	total: 2.26s	remaining: 29.8s
353:	learn: 0.2300628	total: 2.27s	remaining: 29.8s
354:	learn: 0.2294724	total: 2.28s	remaining: 29.8s
355:	learn: 0.2290593	total: 2.29s	remaining: 29.9s
356:	learn: 0.2284444	total: 2.3s	remaining: 29.9s
357:	learn: 0.2278572	total: 2.31s	remaining: 29.9s
358:	learn: 0.2272944	total: 2.31s	remaining: 29.9s
359:	learn: 0.2265061	total: 2.32s	remaining: 29.9s
360:	learn: 0.2256735	total: 2.33s	remaining: 29.9s
361:	learn: 0.2251160	total: 2.33s	remaining: 29.9s
362:	learn: 0.2245595	total: 2.34s	remaining: 29.9s
363:	learn: 0.2240945	total: 2.34s	remaining: 29.9s
364:	learn: 0.2235183	total: 2.35s	remaining: 29.9s
365:	learn: 0.2232446	total: 2.36s	remaining: 29.9s
366:	learn: 0.2226284	total: 2.37s	remaining: 29.9s
367:	learn: 0.2222229	total: 2.38s	remaining: 29.9s
368:	learn: 0

534:	learn: 0.1534078	total: 3.26s	remaining: 27.2s
535:	learn: 0.1532517	total: 3.26s	remaining: 27.2s
536:	learn: 0.1528769	total: 3.27s	remaining: 27.2s
537:	learn: 0.1525901	total: 3.27s	remaining: 27.2s
538:	learn: 0.1522019	total: 3.28s	remaining: 27.1s
539:	learn: 0.1519250	total: 3.28s	remaining: 27.1s
540:	learn: 0.1515993	total: 3.29s	remaining: 27.1s
541:	learn: 0.1513868	total: 3.29s	remaining: 27.1s
542:	learn: 0.1511284	total: 3.3s	remaining: 27.1s
543:	learn: 0.1509378	total: 3.3s	remaining: 27.1s
544:	learn: 0.1506847	total: 3.31s	remaining: 27s
545:	learn: 0.1503768	total: 3.31s	remaining: 27s
546:	learn: 0.1502561	total: 3.32s	remaining: 27s
547:	learn: 0.1496769	total: 3.32s	remaining: 27s
548:	learn: 0.1493939	total: 3.33s	remaining: 27s
549:	learn: 0.1489615	total: 3.33s	remaining: 27s
550:	learn: 0.1487637	total: 3.34s	remaining: 26.9s
551:	learn: 0.1483522	total: 3.34s	remaining: 26.9s
552:	learn: 0.1480639	total: 3.35s	remaining: 26.9s
553:	learn: 0.1476929	tota

711:	learn: 0.1103522	total: 4.3s	remaining: 25.9s
712:	learn: 0.1102609	total: 4.31s	remaining: 25.9s
713:	learn: 0.1101552	total: 4.32s	remaining: 25.9s
714:	learn: 0.1098526	total: 4.33s	remaining: 26s
715:	learn: 0.1096320	total: 4.34s	remaining: 26s
716:	learn: 0.1092153	total: 4.35s	remaining: 26s
717:	learn: 0.1089993	total: 4.36s	remaining: 26s
718:	learn: 0.1088322	total: 4.36s	remaining: 26s
719:	learn: 0.1087817	total: 4.37s	remaining: 26s
720:	learn: 0.1085495	total: 4.38s	remaining: 26s
721:	learn: 0.1084037	total: 4.39s	remaining: 26s
722:	learn: 0.1082611	total: 4.39s	remaining: 26s
723:	learn: 0.1080691	total: 4.4s	remaining: 26s
724:	learn: 0.1078345	total: 4.41s	remaining: 26s
725:	learn: 0.1076445	total: 4.42s	remaining: 26s
726:	learn: 0.1075413	total: 4.42s	remaining: 26s
727:	learn: 0.1073865	total: 4.43s	remaining: 26s
728:	learn: 0.1072657	total: 4.44s	remaining: 26s
729:	learn: 0.1070748	total: 4.45s	remaining: 26s
730:	learn: 0.1068576	total: 4.45s	remaining: 

...

4269:	learn: 0.0106365	total: 24.9s	remaining: 4.25s
4270:	learn: 0.0106336	total: 24.9s	remaining: 4.25s
4271:	learn: 0.0106287	total: 24.9s	remaining: 4.24s
4272:	learn: 0.0106238	total: 24.9s	remaining: 4.24s
4273:	learn: 0.0106211	total: 24.9s	remaining: 4.23s
4274:	learn: 0.0106174	total: 24.9s	remaining: 4.22s
4275:	learn: 0.0106142	total: 24.9s	remaining: 4.22s
4276:	learn: 0.0106103	total: 24.9s	remaining: 4.21s
4277:	learn: 0.0106069	total: 24.9s	remaining: 4.21s
4278:	learn: 0.0106049	total: 24.9s	remaining: 4.2s
4279:	learn: 0.0106011	total: 24.9s	remaining: 4.19s
4280:	learn: 0.0105979	total: 24.9s	remaining: 4.19s
4281:	learn: 0.0105910	total: 24.9s	remaining: 4.18s
4282:	learn: 0.0105873	total: 24.9s	remaining: 4.17s
4283:	learn: 0.0105823	total: 24.9s	remaining: 4.17s
4284:	learn: 0.0105798	total: 25s	remaining: 4.16s
4285:	learn: 0.0105762	total: 25s	remaining: 4.16s
4286:	learn: 0.0105737	total: 25s	remaining: 4.15s
4287:	learn: 0.0105709	total: 25s	remaining: 4.14s
42

4436:	learn: 0.0101282	total: 25.6s	remaining: 3.24s
4437:	learn: 0.0101238	total: 25.6s	remaining: 3.24s
4438:	learn: 0.0101197	total: 25.6s	remaining: 3.23s
4439:	learn: 0.0101168	total: 25.6s	remaining: 3.23s
4440:	learn: 0.0101135	total: 25.6s	remaining: 3.22s
4441:	learn: 0.0101098	total: 25.6s	remaining: 3.21s
4442:	learn: 0.0101079	total: 25.6s	remaining: 3.21s
4443:	learn: 0.0101060	total: 25.6s	remaining: 3.2s
4444:	learn: 0.0101048	total: 25.6s	remaining: 3.19s
4445:	learn: 0.0101025	total: 25.6s	remaining: 3.19s
4446:	learn: 0.0101002	total: 25.6s	remaining: 3.18s
4447:	learn: 0.0100962	total: 25.6s	remaining: 3.18s
4448:	learn: 0.0100935	total: 25.6s	remaining: 3.17s
4449:	learn: 0.0100900	total: 25.6s	remaining: 3.17s
4450:	learn: 0.0100874	total: 25.6s	remaining: 3.16s
4451:	learn: 0.0100848	total: 25.6s	remaining: 3.15s
4452:	learn: 0.0100828	total: 25.6s	remaining: 3.15s
4453:	learn: 0.0100781	total: 25.6s	remaining: 3.14s
4454:	learn: 0.0100765	total: 25.6s	remaining: 

4611:	learn: 0.0096528	total: 26.2s	remaining: 2.21s
4612:	learn: 0.0096504	total: 26.3s	remaining: 2.2s
4613:	learn: 0.0096481	total: 26.3s	remaining: 2.2s
4614:	learn: 0.0096452	total: 26.3s	remaining: 2.19s
4615:	learn: 0.0096423	total: 26.3s	remaining: 2.18s
4616:	learn: 0.0096390	total: 26.3s	remaining: 2.18s
4617:	learn: 0.0096360	total: 26.3s	remaining: 2.17s
4618:	learn: 0.0096347	total: 26.3s	remaining: 2.17s
4619:	learn: 0.0096331	total: 26.3s	remaining: 2.16s
4620:	learn: 0.0096304	total: 26.3s	remaining: 2.16s
4621:	learn: 0.0096272	total: 26.3s	remaining: 2.15s
4622:	learn: 0.0096256	total: 26.3s	remaining: 2.15s
4623:	learn: 0.0096236	total: 26.3s	remaining: 2.14s
4624:	learn: 0.0096222	total: 26.3s	remaining: 2.13s
4625:	learn: 0.0096205	total: 26.3s	remaining: 2.13s
4626:	learn: 0.0096187	total: 26.3s	remaining: 2.12s
4627:	learn: 0.0096169	total: 26.3s	remaining: 2.12s
4628:	learn: 0.0096151	total: 26.4s	remaining: 2.11s
4629:	learn: 0.0096135	total: 26.4s	remaining: 2

4785:	learn: 0.0092301	total: 27.5s	remaining: 1.23s
4786:	learn: 0.0092271	total: 27.5s	remaining: 1.22s
4787:	learn: 0.0092250	total: 27.5s	remaining: 1.22s
4788:	learn: 0.0092219	total: 27.5s	remaining: 1.21s
4789:	learn: 0.0092185	total: 27.5s	remaining: 1.21s
4790:	learn: 0.0092166	total: 27.5s	remaining: 1.2s
4791:	learn: 0.0092142	total: 27.5s	remaining: 1.19s
4792:	learn: 0.0092117	total: 27.5s	remaining: 1.19s
4793:	learn: 0.0092082	total: 27.5s	remaining: 1.18s
4794:	learn: 0.0092059	total: 27.5s	remaining: 1.18s
4795:	learn: 0.0092034	total: 27.5s	remaining: 1.17s
4796:	learn: 0.0092021	total: 27.5s	remaining: 1.17s
4797:	learn: 0.0091996	total: 27.6s	remaining: 1.16s
4798:	learn: 0.0091966	total: 27.6s	remaining: 1.15s
4799:	learn: 0.0091937	total: 27.6s	remaining: 1.15s
4800:	learn: 0.0091914	total: 27.6s	remaining: 1.14s
4801:	learn: 0.0091893	total: 27.6s	remaining: 1.14s
4802:	learn: 0.0091864	total: 27.6s	remaining: 1.13s
4803:	learn: 0.0091853	total: 27.6s	remaining: 

4948:	learn: 0.0088443	total: 28.2s	remaining: 290ms
4949:	learn: 0.0088436	total: 28.2s	remaining: 285ms
4950:	learn: 0.0088421	total: 28.2s	remaining: 279ms
4951:	learn: 0.0088404	total: 28.2s	remaining: 273ms
4952:	learn: 0.0088380	total: 28.2s	remaining: 267ms
4953:	learn: 0.0088364	total: 28.2s	remaining: 262ms
4954:	learn: 0.0088346	total: 28.2s	remaining: 256ms
4955:	learn: 0.0088321	total: 28.2s	remaining: 250ms
4956:	learn: 0.0088311	total: 28.2s	remaining: 245ms
4957:	learn: 0.0088295	total: 28.2s	remaining: 239ms
4958:	learn: 0.0088268	total: 28.2s	remaining: 233ms
4959:	learn: 0.0088243	total: 28.2s	remaining: 227ms
4960:	learn: 0.0088216	total: 28.2s	remaining: 222ms
4961:	learn: 0.0088193	total: 28.2s	remaining: 216ms
4962:	learn: 0.0088175	total: 28.2s	remaining: 210ms
4963:	learn: 0.0088144	total: 28.2s	remaining: 205ms
4964:	learn: 0.0088127	total: 28.2s	remaining: 199ms
4965:	learn: 0.0088104	total: 28.2s	remaining: 193ms
4966:	learn: 0.0088087	total: 28.2s	remaining:

In [36]:
cfc.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'MultiClass',
 'iterations': 1000,
 'sampling_frequency': 'PerTree',
 'leaf_estimation_method': 'Newton',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'model_size_reg': 0.5,
 'use_best_model': False,
 'class_names': [1, 2, 3, 4],
 'random_seed': 0,
 'depth': 6,
 'border_count': 254,
 'bagging_temperature': 1,
 'classes_count': 0,
 'auto_class_weights': 'None',
 'sparse_features_conflict_fraction': 0,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'best_model_min_trees': 1,
 'model_shrink_rate': 0,
 'min_data_in_leaf': 1,
 'loss_function': 'MultiClass',
 'learning_rate': 0.07866200059652328,
 'score_function': 'Cosine',
 'task_type': 'CPU',
 'leaf_estimation_iterations': 1,
 'bootstrap_type': 'Bayesian',

In [51]:
filename = 'catboost_classifier.sav'
pickle.dump(rfc, open(filename, 'wb'))